In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-07-26 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-26 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-26 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-26 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-26 00:00:34


,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-07-26 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-26 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-26 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-26 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-26 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

(13551, 384)

In [7]:
x_data = obtain_path(
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    k=20
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/13551 [00:00<?, ?it/s]

In [8]:
x_data.shape

(13551, 20, 384)

(13551, 20, 384)

In [9]:
x_data[0]

array([[0.00154884, 0.01095446, 0.04541774, ..., 0.02605489, 0.05162514,
        0.0810306 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

array([[0.00154884, 0.01095446, 0.04541774, ..., 0.02605489, 0.05162514,
        0.0810306 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

# Baseline: LSTM classification

In [10]:
num_epochs = 100
hidden_dim_sizes = [100, 200, 300, 384]
num_layers = 1
bidirectional = True
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"

## `history_length=20`

In [ ]:
size = 20
bilstm_history_20_kfold, best_bilstm_history_20_kfold, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/13551 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_20_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_20_focal_2_kfold_best_model.csv


In [ ]:
bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_63264/3632055421.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
100        0.1          0.0001         0.612407  0.506677   0.503848   
                        0.0005         0.628563  0.522077   0.520121   
                        0.0010         0.625057  0.525193   0.519471   
           0.2          0.0001         0.613321  0.508097   0.505083   
                        0.0005         0.624265  0.519125   0.516181   
                        0.0010         0.631672  0.524919   0.521239   
           0.5          0.0001         0.612010  0.500933   0.499883   
                        0.0005         0.623777  0.516144   0.513736   
                        0.0010         0.627252  0.521105   0.517110   
200        0.1          0.0001         0.617863  0.512414   0.509279   
                        0.0005         0.627252  0.523034   0.518883   
                        0.0010         0.623350  0.520857   0.515656   
           0.2          0.0001         0.616400  0.509029   0.506571   
                        0.0005         0.627435  0.522255   0.518674   
                        0.0010         0.625667  0.521971   0.518043   
           0.5          0.0001         0.613870  0.504547   0.503463   
                        0.0005         0.624447  0.519640   0.515941   
                        0.0010         0.625210  0.519517   0.515721   
300        0.1          0.0001         0.610029  0.500532   0.498704   
                        0.0005         0.628685  0.523874   0.520024   
                        0.0010         0.618107  0.518063   0.512110   
           0.2          0.0001         0.608535  0.502710   0.499589   
                        0.0005         0.627831  0.520011   0.517531   
                        0.0010         0.621917  0.516401   0.512387   
           0.5          0.0001         0.602743  0.497908   0.494482   
                        0.0005         0.626551  0.518844   0.515233   
                        0.0010         0.628014  0.522885   0.518524   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
100        0.1          0.0001         0.512610        0.694102  0.632685   
                        0.0005         0.528400        0.709815  0.645129   
                        0.0010         0.535415        0.701444  0.641899   
           0.2          0.0001         0.514120        0.694908  0.633442   
                        0.0005         0.525718        0.708025  0.644915   
                        0.0010         0.531097        0.706167  0.641500   
           0.5          0.0001         0.504823        0.696475  0.632769   
                        0.0005         0.521998        0.705741  0.643020   
                        0.0010         0.527650        0.706704  0.645492   
200        0.1          0.0001         0.518936        0.695020  0.630540   
                        0.0005         0.530207        0.706883  0.645321   
                        0.0010         0.529677        0.699452  0.637199   
           0.2          0.0001         0.514539        0.695400  0.630963   
                        0.0005         0.529053        0.708025  0.645596   
                        0.0010         0.529882        0.699071  0.636409   
           0.5          0.0001         0.509304        0.695870  0.633157   
                        0.0005         0.526898        0.708786  0.647005   
                        0.0010         0.526603        0.701466  0.638924   
300        0.1          0.0001         0.505036        0.694415  0.631739   
                        0.0005         0.530664        0.703302  0.639256   
                        0.0010         0.528012        0.694639  0.633151   
           0.2          0.0001         0.509010        0.691729  0.630491   
                        0.0005         0.525657        0.705204  0.639755   
                        0

In [ ]:
best_bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.613474,0.518238,"[0.7373965387509405, 0.46293969849246236, 0.35...",0.512221,"[0.7682257643062451, 0.4697259400892288, 0.298...",0.533612,"[0.7089462261876055, 0.4563467492260062, 0.435...",None,0.703078,...,True,200,0.5,0.0005,0,focal,2,True,5,64
0,None,0.633440,0.523847,"[0.755064903554531, 0.47780261897157456, 0.338...",0.521692,"[0.759765625, 0.49340369393139843, 0.311907270...",0.528014,"[0.7504219918013021, 0.4631578947368421, 0.370...",None,0.709121,...,True,200,0.5,0.0005,1,focal,2,True,5,64
0,None,0.629325,0.523630,"[0.748460211874846, 0.4859028194361128, 0.3365...",0.519623,"[0.7650465877612692, 0.4712041884816754, 0.322...",0.528605,"[0.7325777670605257, 0.5015479876160991, 0.351...",None,0.708786,...,True,200,0.5,0.0005,12,focal,2,True,5,64
0,None,0.624447,0.516735,"[0.7453793774319066, 0.4714926329276105, 0.333...",0.514378,"[0.7517782683345597, 0.48838752488387527, 0.30...",0.521760,"[0.7390884977091874, 0.45572755417956656, 0.37...",None,0.713151,...,True,200,0.5,0.0005,123,focal,2,True,5,64
0,None,0.621552,0.515750,"[0.7437975927290592, 0.47272727272727266, 0.33...",0.511789,"[0.757947434292866, 0.47873015873015873, 0.298...",0.522501,"[0.7301663853387992, 0.46687306501547987, 0.37...",None,0.709793,...,True,200,0.5,0.0005,1234,focal,2,True,5,64


In [ ]:
best_bilstm_history_20_kfold["f1"].mean()

0.5196401193957776

In [25]:
best_bilstm_history_20_kfold["precision"].mean()

0.5159406031839908

In [26]:
best_bilstm_history_20_kfold["recall"].mean()

0.5268984931980192

In [ ]:
np.stack(best_bilstm_history_20_kfold["f1_scores"]).mean(axis=0)

array([0.74601972, 0.47417301, 0.33872762])

In [27]:
np.stack(best_bilstm_history_20_kfold["precision_scores"]).mean(axis=0)

array([0.76055274, 0.4802903 , 0.30697877])

In [28]:
np.stack(best_bilstm_history_20_kfold["recall_scores"]).mean(axis=0)

array([0.73224017, 0.46873065, 0.37972466])

## `history_length=50`

In [ ]:
size = 50
bilstm_history_50_kfold, best_bilstm_history_50_kfold, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_50_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_50_focal_2_kfold_best_model.csv


In [ ]:
bilstm_history_50_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_63264/1676909037.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_50_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
100        0.1          0.0001         0.608718  0.510673   0.506040   
                        0.0005         0.625301  0.519032   0.516269   
                        0.0010         0.629386  0.526800   0.522297   
           0.2          0.0001         0.609694  0.510272   0.506126   
                        0.0005         0.615912  0.514114   0.509647   
                        0.0010         0.625179  0.523661   0.518487   
           0.5          0.0001         0.607926  0.506125   0.502460   
                        0.0005         0.618168  0.514381   0.510492   
                        0.0010         0.625332  0.519226   0.515596   
200        0.1          0.0001         0.610974  0.511603   0.507157   
                        0.0005         0.624143  0.519680   0.514791   
                        0.0010         0.627039  0.521281   0.517620   
           0.2          0.0001         0.613870  0.511419   0.507938   
                        0.0005         0.626094  0.522935   0.517884   
                        0.0010         0.630605  0.524702   0.521288   
           0.5          0.0001         0.613504  0.507750   0.505573   
                        0.0005         0.627526  0.523980   0.519601   
                        0.0010         0.627953  0.523238   0.519202   
300        0.1          0.0001         0.609724  0.504283   0.500759   
                        0.0005         0.633013  0.525281   0.522219   
                        0.0010         0.625880  0.519493   0.517458   
           0.2          0.0001         0.610974  0.504668   0.501418   
                        0.0005         0.627526  0.520139   0.516558   
                        0.0010         0.620180  0.517885   0.512694   
           0.5          0.0001         0.606036  0.499666   0.496656   
                        0.0005         0.626459  0.522038   0.517737   
                        0.0010         0.623381  0.520245   0.514991   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
100        0.1          0.0001         0.520933        0.708226  0.646920   
                        0.0005         0.525402        0.720134  0.658900   
                        0.0010         0.536122        0.719127  0.656923   
           0.2          0.0001         0.519767        0.707890  0.645526   
                        0.0005         0.523520        0.718747  0.660247   
                        0.0010         0.532691        0.718075  0.658149   
           0.5          0.0001         0.514137        0.706659  0.644925   
                        0.0005         0.522685        0.719284  0.660692   
                        0.0010         0.525902        0.720425  0.658956   
200        0.1          0.0001         0.521786        0.707420  0.646300   
                        0.0005         0.527647        0.719239  0.659057   
                        0.0010         0.529319        0.715658  0.652827   
           0.2          0.0001         0.520032        0.707465  0.645584   
                        0.0005         0.530792        0.717336  0.655644   
                        0.0010         0.530687        0.718097  0.653640   
           0.5          0.0001         0.514099        0.707868  0.644542   
                        0.0005         0.531809        0.717426  0.657122   
                        0.0010         0.531445        0.717202  0.655915   
300        0.1          0.0001         0.510539        0.708316  0.646432   
                        0.0005         0.530722        0.719597  0.656841   
                        0.0010         0.527073        0.712860  0.648875   
           0.2          0.0001         0.510778        0.709345  0.647542   
                        0.0005         0.525104        0.720515  0.658368   
                        0

In [ ]:
best_bilstm_history_50_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.623076,0.524026,"[0.7431829449677739, 0.47854682117131225, 0.35...",0.518575,"[0.7646008671257333, 0.4841571609632446, 0.306...",0.534669,"[0.7229322401736195, 0.47306501547987617, 0.40...",None,0.720761,...,True,100,0.5,0.0005,0,focal,2,True,5,64
0,None,0.607834,0.511511,"[0.726281075596144, 0.4937537180249852, 0.3144...",0.505044,"[0.7661225582017661, 0.47510017172295366, 0.27...",0.524507,"[0.6903785869303111, 0.5139318885448917, 0.369...",None,0.715725,...,True,100,0.5,0.0005,1,focal,2,True,5,64
0,None,0.628868,0.517006,"[0.74906094753423, 0.479874213836478, 0.322084...",0.515320,"[0.7528007793472966, 0.48753993610223645, 0.30...",0.519410,"[0.7453580901856764, 0.4724458204334365, 0.340...",None,0.720425,...,True,100,0.5,0.0005,12,focal,2,True,5,64
0,None,0.607682,0.507054,"[0.7317497823653775, 0.46368537900412304, 0.32...",0.502979,"[0.7555213148433487, 0.4752925877763329, 0.278...",0.518350,"[0.7094285025319508, 0.45263157894736844, 0.39...",None,0.711696,...,True,100,0.5,0.0005,123,focal,2,True,5,64
0,None,0.623381,0.512310,"[0.7478197674418606, 0.45861513687600647, 0.33...",0.510542,"[0.7512776831345827, 0.4778523489932886, 0.302...",0.516489,"[0.7443935374969858, 0.4408668730650155, 0.364...",None,0.727812,...,True,100,0.5,0.0005,1234,focal,2,True,5,64


In [ ]:
best_bilstm_history_50_kfold["f1"].mean()

0.5143814831518355

In [21]:
best_bilstm_history_50_kfold["precision"].mean()

0.5104918274487381

In [22]:
best_bilstm_history_50_kfold["recall"].mean()

0.5226850458821458

In [ ]:
np.stack(best_bilstm_history_50_kfold["f1_scores"]).mean(axis=0)

array([0.7396189 , 0.47489505, 0.32863049])

In [23]:
np.stack(best_bilstm_history_50_kfold["precision_scores"]).mean(axis=0)

array([0.75806464, 0.47998844, 0.2934224 ])

In [24]:
np.stack(best_bilstm_history_50_kfold["recall_scores"]).mean(axis=0)

array([0.72249819, 0.47058824, 0.37496871])